In [1]:
import sklearn
import pandas as pd
import numpy as np
import numba as nb
import pickle 
import os
import functools as ft
import swifter

# data loading

In [5]:
data_wd = "/data/datasets/Tianchen/2022_three_cancer_Yu_1FL_data/data"

In [6]:
#the data below follow PCORnet common data model
#https://pcornet.org/wp-content/uploads/2022/01/PCORnet-Common-Data-Model-v60-2020_10_221.pdf

death = pd.read_csv(data_wd+"/DEATH.csv",header=0,skiprows=[1])
demographics = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])
diagnosis = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
lab_result_cm = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])

#The data below may not be needed for this project, at least at this time
####################################################################################
#procedure = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
#dispensing = pd.read_csv(data_wd+"/DISPENSING.csv",header=0,skiprows=[1])
#med_admin = pd.read_csv(data_wd+"/MED_ADMIN.csv",header=0,skiprows=[1])
#vital = pd.read_csv(data_wd+"/VITAL.csv",header=0,skiprows=[1])
#encounter = pd.read_csv(data_wd+"/ENCOUNTER.csv",header=0,skiprows=[1])
#prescribing = pd.read_csv(data_wd+"/PRESCRIBING.csv",header=0,skiprows=[1])

/tmp/ipykernel_18264/962649024.py:3: DtypeWarning: Columns (7,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  diagnosis = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
/tmp/ipykernel_18264/962649024.py:4: DtypeWarning: Columns (9,10,12,13,17,21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_result_cm = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])


In [7]:
#the data below is the index date of cancer onset, kindly provided by Tianchen
index_date = pd.read_csv('/data/datasets/Tianchen/2022_three_cancer_Yu_1FL_data/f_cancer_yu_index_date.csv')

# Cohort subsetting

*The summary statistics number were used to generate the numbers in statistic spreadsheet*

## cancer ICD definition

In [8]:
#cancer icd, provided in the data request form 
lung_cancer_icd = ['162.2',
                   '162.3',
                   '162.4',
                   '162.5',
                   '162.8',
                   '162.9',
                   'C34',
                   'C34.0',
                   'C34.01','C34.02','C34.00',
                   'C34.1','C34.10','C34.11','C34.12',
                   'C34.2',
                   'C34.3','C34.30','C34.31','C34.32',
                   'C34.8','C34.80','C34.82','C34.81',
                   'C34.9','C34.90','C34.91','C34.92',
                   '1622',
                   '1623',
                   '1624',
                   '1625',
                   '1628',
                   '1629',
                   'C34',
                   'C340',
                   'C3401','C3402','C3400',
                   'C341','C3410','C3411','C3412',
                   'C342',
                   'C343','C3430','C3431','C3432',
                   'C348','C3480','C3482','C3481',
                   'C349','C3490','C3491','C3492']

In [9]:
colorectal_cancer_icd = ['153','153.0','153.1','153.2','153.3','153.4','153.5','153.6','153.7','153.8','153.9',
                         '154','154.0','154.1','154.2','154.3','154.8',
                         '159.0',
                         'C18','C18.0','C18.1','C18.2','C18.3','C18.4','C18.5','C18.6','C18.7','C18.8','C18.9',
                         'C19',
                         'C20',
                         'C21','C21.0','C21.1','C21.2','C21.8',
                         'C26.0',
                        '153','1530','1531','1532','1533','1534','1535','1536','1537','1538','1539',
                         '154','1540','1541','1542','1543','1548',
                         '1590',
                         'C18','C180','C181','C182','C183','C184','C185','C186','C187','C188','C189',
                         'C19',
                         'C20',
                         'C21','C210','C211','C212','C218',
                         'C260']

In [10]:
breast_cancer_icd = ['174','174.0','174.1','174.2','174.3','174.4','174.5','174.6','174.8','174.9',
                     '175','175.0','175.9',
                     'C50',
                     'C50.0',
                     'C50.01','C50.011','C50.012','C50.019',
                     'C50.02','C50.021','C50.022','C50.029',
                     'C50.1',
                     'C50.11','C50.111','C50.112','C50.119',
                     'C50.12','C50.121','C50.122','C50.129',
                     'C50.2',
                     'C50.21','C50.211','C50.212','C50.219',
                     'C50.22','C50.221','C50.222','C50.229',
                     'C50.3',
                     'C50.31','C50.311','C50.312','C50.319',
                     'C50.32','C50.321','C50.322','C50.329',
                     'C50.4',
                     'C50.41','C50.411','C50.412','C50.419',
                     'C50.42','C50.421','C50.422','C50.429',
                     'C50.5',
                     'C50.51','C50.511','C50.512','C50.519',
                     'C50.52','C50.521','C50.522','C50.529',
                     'C50.6',
                     'C50.61','C50.611','C50.612','C50.619',
                     'C50.62','C50.621','C50.622','C50.629',
                     'C50.8',
                     'C50.81','C50.811','C50.812','C50.819',
                     'C50.82','C50.821','C50.822','C50.829',
                     'C50.9',
                     'C50.91','C50.911','C50.912','C50.919',
                     'C50.92','C50.921','C50.922','C50.929',
                     
                     '174','1740','1741','1742','1743','1744','1745','1746','1748','1749',
                     '175','1750','1759',
                     'C50',
                     'C500',
                     'C5001','C50011','C50012','C50019',
                     'C5002','C50021','C50022','C50029',
                     'C501',
                     'C5011','C50111','C50112','C50119',
                     'C5012','C50121','C50122','C50129',
                     'C502',
                     'C5021','C50211','C50212','C50219',
                     'C5022','C50221','C50222','C50229',
                     'C503',
                     'C5031','C50311','C50312','C50319',
                     'C5032','C50321','C50322','C50329',
                     'C504',
                     'C5041','C50411','C50412','C50419',
                     'C5042','C50421','C50422','C50429',
                     'C505',
                     'C5051','C50511','C50512','C50519',
                     'C5052','C50521','C50522','C50529',
                     'C506',
                     'C5061','C50611','C50612','C50619',
                     'C5062','C50621','C50622','C50629',
                     'C508',
                     'C5081','C50811','C50812','C50819',
                     'C5082','C50821','C50822','C50829',
                     'C509',
                     'C5091','C50911','C50912','C50919',
                     'C5092','C50921','C50922','C50929'
                    ]

In [11]:
all3_cancer_icd = np.concatenate((lung_cancer_icd,colorectal_cancer_icd,breast_cancer_icd),axis=0)

## cohort 1: all cancer patients

*This step serves as data quality check. The above ICD codes are from the data extraction form. Ideally, this entire cancer cohort equal to the whole dataset we have.*

In [12]:
all3_patient = diagnosis.loc[diagnosis['DX'].isin(all3_cancer_icd)][['DX','ID']].drop_duplicates().reset_index(drop=True)
#all3_patient.groupby('DX').size()

In [13]:
#total number of patients by grouping size, filtered by cancer icd codes
all3_patient[['ID']].drop_duplicates().shape

(261838, 1)

In [15]:
#total number of patients in `diagnosis` table
len(diagnosis[['ID']].drop_duplicates())

261841

*clearly, there are three patients mismatch. The below subsection check the reason why.*

In [16]:
all3_patient_TMP = diagnosis.loc[~diagnosis['ID'].isin(all3_patient['ID'])][['ID']].drop_duplicates().reset_index(drop=True)

In [17]:
all3_patient_TMP

,ID
0,03Nov2022IRB20200313700171589
1,03Nov2022IRB20200313700150936
2,03Nov2022IRB20200313700204041


In [18]:
#get the ICD codes of these 3 mismatched patients
p1 = diagnosis.loc[diagnosis['ID']=='03Nov2022IRB20200313700204041']['DX'].drop_duplicates().values

In [19]:
p2 = diagnosis.loc[diagnosis['ID']=='03Nov2022IRB20200313700171589']['DX'].drop_duplicates().values

In [20]:
p3 = diagnosis.loc[diagnosis['ID']=='03Nov2022IRB20200313700150936']['DX'].drop_duplicates().values

In [21]:
p1

array(['K62.5', 'I12.9', 'E77.4', 'J44.9', 'N40.0', 'C20.YC', 'D64.9',
       'E11.9', 'K42.9', 'F17.210', 'N20.0', 'N18.9', 'R42', 'I10',
       'I49.3', 'R06.00'], dtype=object)

In [22]:
p2

array(['305.60', '300.9', '995.90', '298.9', '270.6', '977.9', '518.81',
       '276.2', 'F11.20', '659.71', '998.13', '781.0', 'V22.1', '305.1',
       '304.00', 'V22.2', 'V23.49', '649.03', '304.01', 'V23.7', '287.5',
       '649.33', 'V58.69', '285.9', 'V02.51', '648.91', '648.21',
       '649.01', 'V27.0', '645.21', '648.31', '162.59', '782.3', '292.9',
       '655.83', 'V22.0', 'O34.29', 'F19.20', 'Z22.338', 'Z34.80',
       'Z34.93', 'O09.622', 'B18.1', 'B19.20', 'O26.613', 'O35.5XX0',
       'Z33.1', 'Z13.828', 'Z83.3', 'L02.91', 'O42.013', 'O32.1XX0',
       'O99.324', 'O34.219', 'Z37.0', 'R10.30', 'Z3A.39', 'F17.210',
       'O99.334', 'F43.10', 'H00.011', 'L03.213', 'H00.016'], dtype=object)

In [23]:
p3

array(['Z12.4', 'I49.9', 'I48.92', 'E03.9', 'Z01.00', 'Z12.11', 'B35.1',
       'S86.819A', 'D68.69', 'K31.6', 'I10', 'E78.2', 'M54.5', 'G89.4',
       'M81.0', 'J44.9', 'Z23', 'Z87.39', 'E05.90', 'Z78.0', 'Z87.891',
       'Z79.899', 'S82.035A', 'M17.12', 'Z20.822', 'I25.2', 'M06.9',
       'Z13.29', 'Z12.31', 'L30.9', 'K46.9', 'M15.0', 'L98.8', 'R19.7',
       'N63.0', 'Z51.81', 'Z00.00', 'N20.0', 'R31.9', 'R31.0', 'M54.16',
       'M25.50', 'R53.83', 'K43.6', 'K63.2', 'K56.7', 'N17.9', 'R53.81',
       'R10.9', 'Z46.82', 'S22.009A', 'E66.9', 'N39.0', 'K56.609',
       'R00.2', 'I50.9', 'R06.02', 'N13.6', 'I12.9', 'N20.2', 'N28.1',
       'N18.4', 'D64.9', 'E66.01', 'E11.22', 'Z68.31', 'I48.91', 'Z90.49',
       'K21.9', 'Z93.6', 'Z87.442', 'R59.0', 'G89.29', 'Z01.419', 'R01.1',
       'H10.13', 'H35.372', 'H18.51', 'H40.1131', 'I11.0', 'E11.9',
       'M19.041', 'Z68.32', 'G47.33', 'M19.042', 'H40.9', 'D53.9',
       'N18.3', 'R73.09', 'R50.9', 'N20.1', 'R31.29', 'N63.21', 'N64.89',

In [24]:
#The below codes check whether there are some overlapping between the 3 mismatches
p1[np.isin(p1,p2)]

array(['F17.210'], dtype=object)

In [25]:
p1[np.isin(p1,p3)]

array(['I12.9', 'J44.9', 'D64.9', 'E11.9', 'K42.9', 'F17.210', 'N20.0',
       'I10', 'R06.00'], dtype=object)

In [26]:
p2[np.isin(p2,p3)]

array(['305.1', 'V58.69', '285.9', '782.3', 'F17.210'], dtype=object)

***number of patients in the patients identified by icd, is not equal to the total number of patients in the whole data sets. Maybe there are some other icd used by Tianchen to extract the study cohort besides the icd used in data request form?***

***Conclusion: only 3 mismatches should be acceptable, so let's disregard them for now.***

In [27]:
#all types of diagnosis ICD codes
len(diagnosis[['DX']].drop_duplicates())

61545

In [28]:
#all patient in `lab` table
len(lab_result_cm[['ID']].drop_duplicates())

152014

In [29]:
#check table consistency, `patients`
np.mean(diagnosis['ID'].isin(demographics['ID']))

1.0

In [30]:
#check table consistency, `lab`
np.mean(lab_result_cm['ID'].isin(demographics['ID']))

1.0

## cohort 2: lung cancer

In [31]:
lung_patient = diagnosis.loc[diagnosis['DX'].isin(lung_cancer_icd)][['DX','ID']].drop_duplicates().reset_index(drop=True)
#lung_patient.groupby('DX').size()

In [32]:
len(lung_patient['ID'].drop_duplicates())

89222

## cohort 3: colorectal cancer

In [33]:
colorectal_patient = diagnosis.loc[diagnosis['DX'].isin(colorectal_cancer_icd)][['DX','ID']].drop_duplicates().reset_index(drop=True)
#colorectal_patient.groupby('DX').size()

In [34]:
len(colorectal_patient['ID'].drop_duplicates())

68422

## cohort 4: breast cancer

In [35]:
breast_patient = diagnosis.loc[diagnosis['DX'].isin(breast_cancer_icd)][['DX','ID']].drop_duplicates().reset_index(drop=True)
#breast_patient.groupby('DX').size()

In [36]:
len(breast_patient['ID'].drop_duplicates())

117092

# Death date cleaning

In [40]:
print(len(death['ID'].drop_duplicates()))
print(sum(death['DEATH_DATE'].isna()))
death.loc[death['DEATH_DATE'].isna()]

20093
105


,ID,DEATH_DATE,DEATH_DATE_IMPUTE,DEATH_SOURCE,DEATH_MATCH_CONFIDENCE,SOURCE_masked
114,03Nov2022IRB20200313700011522,NaN,NI,L,NI,source_11
174,03Nov2022IRB20200313700017704,NaN,NI,L,NI,source_3
623,03Nov2022IRB20200313700063673,NaN,NI,L,NI,source_3
926,03Nov2022IRB20200313700098161,NaN,NI,L,NI,source_3
1275,03Nov2022IRB20200313700134247,NaN,NI,L,NI,source_3
...,...,...,...,...,...,...
19634,03Nov2022IRB20200313700217574,NaN,NI,L,NI,source_3
19719,03Nov2022IRB20200313700224807,NaN,NI,D,NI,source_3
19725,03Nov2022IRB20200313700224849,NaN,NI,L,NI,source_3
19962,03Nov2022IRB20200313700252121,NaN,NI,L,NI,source_11


## fill missing values

In [53]:
#fill na with large number as a placeholder
death=death.fillna('3099-12-31')

## drop duplicates, caused by different data source

In [54]:
#all have only 1 death date
death[['ID','DEATH_DATE']].groupby('ID').count().sort_values('DEATH_DATE',ascending=False).reset_index()

,ID,DEATH_DATE
0,03Nov2022IRB20200313700058952,2
1,03Nov2022IRB20200313700220221,2
2,03Nov2022IRB20200313700003785,2
3,03Nov2022IRB20200313700012221,2
4,03Nov2022IRB20200313700252007,2
...,...,...
20088,03Nov2022IRB20200313700087767,1
20089,03Nov2022IRB20200313700087750,1
20090,03Nov2022IRB20200313700087748,1
20091,03Nov2022IRB20200313700087742,1


In [55]:
#take the earlier death date
death_date = death[['ID','DEATH_DATE']].groupby('ID').agg({'DEATH_DATE':'min'}).reset_index()

In [58]:
death_date.loc[death_date['DEATH_DATE']=='3099-12-31']

,ID,DEATH_DATE
222,03Nov2022IRB20200313700002790,3099-12-31
334,03Nov2022IRB20200313700004133,3099-12-31
913,03Nov2022IRB20200313700011522,3099-12-31
948,03Nov2022IRB20200313700011920,3099-12-31
976,03Nov2022IRB20200313700012401,3099-12-31
...,...,...
19549,03Nov2022IRB20200313700254437,3099-12-31
19601,03Nov2022IRB20200313700255293,3099-12-31
19716,03Nov2022IRB20200313700256826,3099-12-31
19742,03Nov2022IRB20200313700257150,3099-12-31


In [65]:
#replace placeholder back to N.A. But the reason why there's n.a. death date in death dataset is unknow.
death_date['DEATH_DATE'] = np.where(death_date['DEATH_DATE']=='3099-12-31',np.nan,death_date['DEATH_DATE'])
print(len(death_date.loc[death_date['DEATH_DATE']=='3099-12-31']))

0


## match cleaned death date with cohorts

In [66]:
lung_patient = pd.merge(pd.merge(lung_patient,death_date,on='ID',how='left'),index_date,on='ID',how='left')

In [67]:
colorectal_patient = pd.merge(pd.merge(colorectal_patient,death_date,on='ID',how='left'),index_date,on='ID',how='left')

In [68]:
breast_patient = pd.merge(pd.merge(breast_patient,death_date,on='ID',how='left'),index_date,on='ID',how='left')

## calculate the number of patients die after 5 years.(used in the execel spreadsheet)

In [72]:
lung_death = lung_patient[['ID','index_date','DEATH_DATE']].drop_duplicates().reset_index()

In [73]:
colorectal_death = colorectal_patient[['ID','index_date','DEATH_DATE']].drop_duplicates().reset_index()

In [74]:
breast_death = breast_patient[['ID','index_date','DEATH_DATE']].drop_duplicates().reset_index()

In [75]:
#add an index column of 5 years
lung_death['year5']=pd.to_datetime(lung_death['index_date'])+pd.DateOffset(years=5)
colorectal_death['year5']=pd.to_datetime(colorectal_death['index_date'])+pd.DateOffset(years=5)
breast_death['year5']=pd.to_datetime(breast_death['index_date'])+pd.DateOffset(years=5)

In [83]:
print('number of lung cancer patient who died: '+str(len(lung_death.loc[~lung_death['DEATH_DATE'].isna()])))

number of lung cancer patient who died: 11326


In [85]:
print('number of colorectal cancer patient who died: '+str(len(colorectal_death.loc[~colorectal_death['DEATH_DATE'].isna()])))

number of colorectal cancer patient who died: 5332


In [86]:
print('number of breast cancer patient who died: '+str(len(breast_death.loc[~breast_death['DEATH_DATE'].isna()])))

number of breast cancer patient who died: 4908


In [87]:
print('number of lung cancer patient who died with in 5 years: '+str(len(lung_death.loc[(~lung_death['DEATH_DATE'].isna())&(lung_death['year5']>=pd.to_datetime(lung_death['DEATH_DATE']))])))

number of lung cancer patient who died with in 5 years: 10652


In [88]:
print('number of colorectal cancer patient who died with in 5 years: '+str(len(colorectal_death.loc[(~colorectal_death['DEATH_DATE'].isna())&(colorectal_death['year5']>=pd.to_datetime(colorectal_death['DEATH_DATE']))])))

number of colorectal cancer patient who died with in 5 years: 4768


In [89]:
print('number of colorectal cancer patient who died with in 5 years: '+str(len(breast_death.loc[(~breast_death['DEATH_DATE'].isna())&(breast_death['year5']>=pd.to_datetime(breast_death['DEATH_DATE']))])))

number of colorectal cancer patient who died with in 5 years: 3951


# Preprocessing - Pair demographics & index date to lab results

In [91]:
## check structure
print(lab_result_cm.columns)
lab_result_cm.head()

Index(['LAB_RESULT_CM_ID', 'ID', 'ENCOUNTERID', 'SPECIMEN_SOURCE', 'LAB_LOINC',
       'LAB_RESULT_SOURCE', 'LAB_LOINC_SOURCE', 'PRIORITY', 'RESULT_LOC',
       'LAB_PX', 'LAB_PX_TYPE', 'LAB_ORDER_DATE', 'SPECIMEN_DATE',
       'SPECIMEN_TIME', 'RESULT_DATE', 'RESULT_TIME', 'RESULT_QUAL',
       'RESULT_SNOMED', 'RESULT_NUM', 'RESULT_MODIFIER', 'RESULT_UNIT',
       'NORM_RANGE_LOW', 'NORM_MODIFIER_LOW', 'NORM_RANGE_HIGH',
       'NORM_MODIFIER_HIGH', 'ABN_IND', 'SOURCE_masked'],
      dtype='object')


,LAB_RESULT_CM_ID,ID,ENCOUNTERID,SPECIMEN_SOURCE,LAB_LOINC,LAB_RESULT_SOURCE,LAB_LOINC_SOURCE,PRIORITY,RESULT_LOC,LAB_PX,...,RESULT_SNOMED,RESULT_NUM,RESULT_MODIFIER,RESULT_UNIT,NORM_RANGE_LOW,NORM_MODIFIER_LOW,NORM_RANGE_HIGH,NORM_MODIFIER_HIGH,ABN_IND,SOURCE_masked
0,cfhOcbEKSlSqrw28,03Nov2022IRB20200313700112267,cfhOdr8FS1Oooww=,SER_PLAS,2823-3,OD,DW,R,L,LAB17,...,NaN,4.1,EQ,mmol/L,3.3,EQ,5.1,EQ,NI,source_3
1,cfhOcbEKSlSqrwC7,03Nov2022IRB20200313700244847,cfhOdr8ESFCsrgE=,SER_PLAS,2885-2,OD,DW,R,L,LAB17,...,NaN,6.9,EQ,g/dL,6.1,EQ,8.1,EQ,NI,source_3
2,cfhOcbEKSlSqrwe4,03Nov2022IRB20200313700180880,cfhOdr8FS16gogE=,SER_PLAS,10466-1,OD,DW,S,L,LAB15,...,NaN,10.0,EQ,mmol/L,8,EQ,16,EQ,NI,source_3
3,cfhOcbEKSlSqrwG6,03Nov2022IRB20200313700167813,cfhOdr8FSVGoqgc=,BLD,787-2,OD,DW,R,L,LAB123294,...,NaN,86.6,EQ,fL,82.0,EQ,101.0,EQ,NI,source_3
4,cfhOcbEKSlSqrwK7,03Nov2022IRB20200313700193361,cfhOdr8ESlesqw0=,BLD,2339-0,OD,DW,R,L,LAB43790,...,NaN,197.0,EQ,mg/dL,65,EQ,99,EQ,AH,source_3


In [92]:
#total number of LOINC types
len(lab_result_cm[['LAB_LOINC']].drop_duplicates())

4586

## cleaning `demographics` and `index_date`

In [93]:
#pair demographics and index date
demographics = pd.merge(demographics,index_date,left_on='ID',right_on='ID',how='inner')

In [94]:
#keep only those useful ones
demographics = demographics[['ID', 'BIRTH_DATE', 'SEX', 'HISPANIC', 'RACE', 'ZIP_CODE', 'index_date']]

In [95]:
#convert datetime type
demographics['BIRTH_DATE']=pd.to_datetime(demographics['BIRTH_DATE'])
demographics['index_date']=pd.to_datetime(demographics['index_date'])

In [96]:
#truncate zipcode to only 5 characters for uniformity
demographics['ZIP_CODE'] = demographics['ZIP_CODE'].str.slice(0,5)

In [97]:
#define function for calculating ages
def calc_age(born_d,index_d):
   return index_d.year - born_d.year - ((index_d.month, 
                                      index_d.day) < (born_d.month, 
                                                    born_d.day))

In [98]:
#calculate age at diagnosis (index_date provided)
demographics['index_age'] = demographics.swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.index_date), axis=1)

In [99]:
#calculate study range, 6~2 years before index date
demographics['study_range_min'] = demographics['index_date'] - pd.DateOffset(years=6)
demographics['study_range_max'] = demographics['index_date'] - pd.DateOffset(years=2)
demographics['study_range_max_1'] = demographics['index_date'] - pd.DateOffset(years=1)
demographics['study_range_max_05'] = demographics['index_date'] - pd.DateOffset(months=6)

In [101]:
demographics.head(2)

,ID,BIRTH_DATE,SEX,HISPANIC,RACE,ZIP_CODE,index_date,index_age,study_range_min,study_range_max,study_range_max_1,study_range_max_05
0,03Nov2022IRB20200313700102206,1961-10-15,F,Y,05,32312,2014-04-07,52,2008-04-07,2012-04-07,2013-04-07,2013-10-07
1,03Nov2022IRB20200313700159940,1941-10-15,F,N,05,32310,2019-05-25,77,2013-05-25,2017-05-25,2018-05-25,2018-11-25


## start paring `demographics`, `index_date`,`death`, and `lab`

In [130]:
#pair demographics
print(lab_result_cm.shape)
lab_result_cm_paired = pd.merge(lab_result_cm,demographics,left_on='ID',right_on='ID',how='left')
print(lab_result_cm_paired.shape)

(76978078, 27)
(76978078, 38)


In [131]:
lab_result_cm_paired = pd.merge(lab_result_cm_paired,death_date,left_on='ID',right_on='ID',how='left')
print(lab_result_cm_paired.shape)

(76978078, 39)


In [132]:
len(death['ID'])

20117

In [133]:
len(set(death['ID']))

20093

In [134]:
#define a faster version of np.isin(.)
@nb.jit(parallel=True)
def is_in_set_pnb(a, b):
    shape = a.shape
    a = a.ravel()
    n = len(a)
    result = np.full(n, False)
    set_b = set(b)
    for i in nb.prange(n):
        if a[i] in set_b:
            result[i] = True
    return result.reshape(shape)

In [173]:
#tagging. This will be used in figures.
lab_result_cm_paired['IF_any3cancer'] = np.where(is_in_set_pnb(lab_result_cm_paired['ID'],all3_patient['ID']),1,0)
lab_result_cm_paired['BreastCancer'] = np.where(is_in_set_pnb(lab_result_cm_paired['ID'],breast_patient['ID']),1,0)
lab_result_cm_paired['ColorectalCancer'] = np.where(is_in_set_pnb(lab_result_cm_paired['ID'],colorectal_patient['ID']),1,0)
lab_result_cm_paired['LungCancer'] = np.where(is_in_set_pnb(lab_result_cm_paired['ID'],lung_patient['ID']),1,0)

In [136]:
#check consistency
print(lab_result_cm_paired.shape),print(lab_result_cm.shape)
print(lab_result_cm_paired.columns)
lab_result_cm_paired.head(2)

(76978078, 43)
(76978078, 27)
Index(['LAB_RESULT_CM_ID', 'ID', 'ENCOUNTERID', 'SPECIMEN_SOURCE', 'LAB_LOINC',
       'LAB_RESULT_SOURCE', 'LAB_LOINC_SOURCE', 'PRIORITY', 'RESULT_LOC',
       'LAB_PX', 'LAB_PX_TYPE', 'LAB_ORDER_DATE', 'SPECIMEN_DATE',
       'SPECIMEN_TIME', 'RESULT_DATE', 'RESULT_TIME', 'RESULT_QUAL',
       'RESULT_SNOMED', 'RESULT_NUM', 'RESULT_MODIFIER', 'RESULT_UNIT',
       'NORM_RANGE_LOW', 'NORM_MODIFIER_LOW', 'NORM_RANGE_HIGH',
       'NORM_MODIFIER_HIGH', 'ABN_IND', 'SOURCE_masked', 'BIRTH_DATE', 'SEX',
       'HISPANIC', 'RACE', 'ZIP_CODE', 'index_date', 'index_age',
       'study_range_min', 'study_range_max', 'study_range_max_1',
       'study_range_max_05', 'DEATH_DATE', 'IF_any3cancer', 'IF_breastcancer',
       'IF_colorectalcancer', 'IF_lungcancer'],
      dtype='object')


,LAB_RESULT_CM_ID,ID,ENCOUNTERID,SPECIMEN_SOURCE,LAB_LOINC,LAB_RESULT_SOURCE,LAB_LOINC_SOURCE,PRIORITY,RESULT_LOC,LAB_PX,...,index_age,study_range_min,study_range_max,study_range_max_1,study_range_max_05,DEATH_DATE,IF_any3cancer,IF_breastcancer,IF_colorectalcancer,IF_lungcancer
0,cfhOcbEKSlSqrw28,03Nov2022IRB20200313700112267,cfhOdr8FS1Oooww=,SER_PLAS,2823-3,OD,DW,R,L,LAB17,...,55,2006-01-09,2010-01-09,2011-01-09,2011-07-09,NaN,1,1,0,0
1,cfhOcbEKSlSqrwC7,03Nov2022IRB20200313700244847,cfhOdr8ESFCsrgE=,SER_PLAS,2885-2,OD,DW,R,L,LAB17,...,60,2015-12-20,2019-12-20,2020-12-20,2021-06-20,NaN,1,0,0,1


In [137]:
lab_result_cm_paired['IF_any3cancer'].value_counts()

1    76973186
0        4892
Name: IF_any3cancer, dtype: int64

In [138]:
lab_result_cm_paired['BreastCancer'].value_counts()

0    49322307
1    27655771
Name: IF_breastcancer, dtype: int64

In [139]:
lab_result_cm_paired['ColorectalCancer'].value_counts()

0    51996274
1    24981804
Name: IF_colorectalcancer, dtype: int64

In [140]:
lab_result_cm_paired['LungCancer'].value_counts()

0    46776104
1    30201974
Name: IF_lungcancer, dtype: int64

In [141]:
len(set(lab_result_cm_paired.loc[lab_result_cm_paired['IF_any3cancer']!=1]['ID']))

1

In [142]:
#exclude those 3 mismatches aforementioned
lab_result_cm_paired = lab_result_cm_paired.loc[lab_result_cm_paired['IF_any3cancer']==1]

In [160]:
print(lab_result_cm_paired.shape)
print(lab_result_cm.shape)

(76973186, 43)
(76978078, 27)


# Keep only female and male

In [162]:
lab_result_cm_paired = lab_result_cm_paired.loc[is_in_set_pnb(lab_result_cm_paired['SEX'],['F','M'])]

/tmp/ipykernel_18264/714591504.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "is_in_set_pnb" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_18264/714591504.py (4)

File "../../../tmp/ipykernel_18264/714591504.py", line 4:
<source missing, REPL/exec in use?>

  @nb.jit(parallel=True)
/tmp/ipykernel_18264/714591504.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "is_in_set_pnb" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../../../tmp/ipykernel_18264/714591504.py", line 9:
<source missing, REPL/exec in use?>

  @nb.jit(parallel=True)
/home/changyu.yin/.pyenv/versions/miniconda3-latest/envs/rapids-22.12/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "is_in_set_pnb" was compiled in object mode without forceobj=T

In [164]:
print(lab_result_cm_paired.shape)
lab_result_cm_paired.head(2)

(76972453, 43)


,LAB_RESULT_CM_ID,ID,ENCOUNTERID,SPECIMEN_SOURCE,LAB_LOINC,LAB_RESULT_SOURCE,LAB_LOINC_SOURCE,PRIORITY,RESULT_LOC,LAB_PX,...,index_age,study_range_min,study_range_max,study_range_max_1,study_range_max_05,DEATH_DATE,IF_any3cancer,IF_breastcancer,IF_colorectalcancer,IF_lungcancer
0,cfhOcbEKSlSqrw28,03Nov2022IRB20200313700112267,cfhOdr8FS1Oooww=,SER_PLAS,2823-3,OD,DW,R,L,LAB17,...,55,2006-01-09,2010-01-09,2011-01-09,2011-07-09,NaN,1,1,0,0
1,cfhOcbEKSlSqrwC7,03Nov2022IRB20200313700244847,cfhOdr8ESFCsrgE=,SER_PLAS,2885-2,OD,DW,R,L,LAB17,...,60,2015-12-20,2019-12-20,2020-12-20,2021-06-20,NaN,1,0,0,1


# Preprocessing: Age at lab order date

In [157]:
lab_result_cm_paired['LAB_ORDER_DATE']=pd.to_datetime(lab_result_cm_paired['LAB_ORDER_DATE'])

In [168]:
lab_result_cm_paired['lab_age'] = lab_result_cm_paired[['BIRTH_DATE','LAB_ORDER_DATE']].swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.LAB_ORDER_DATE), axis=1)

Dask Apply:   0%|          | 0/192 [00:00<?, ?it/s]

/tmp/ipykernel_18264/744594513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab_result_cm_paired['lab_age'] = lab_result_cm_paired[['BIRTH_DATE','LAB_ORDER_DATE']].swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.LAB_ORDER_DATE), axis=1)


In [169]:
#calc group
lab_result_cm_paired['age_band'] = pd.cut(lab_result_cm_paired.lab_age, \
                                    [-999,20,30,40,50,60,70,80,999],\
                                    labels = ['[0,20)','[20,30)','[30,40)','[40,50)','[50,60)','[60,70)','[70,80)','>=80'],\
                                    include_lowest=True,right=False)
print(lab_result_cm_paired.shape)

(76972453, 45)


/tmp/ipykernel_18264/718505383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab_result_cm_paired['age_band'] = pd.cut(lab_result_cm_paired.lab_age, \


In [174]:
print(lab_result_cm_paired.shape)
print(lab_result_cm_paired.columns)

(76972453, 45)
Index(['LAB_RESULT_CM_ID', 'ID', 'ENCOUNTERID', 'SPECIMEN_SOURCE', 'LAB_LOINC',
       'LAB_RESULT_SOURCE', 'LAB_LOINC_SOURCE', 'PRIORITY', 'RESULT_LOC',
       'LAB_PX', 'LAB_PX_TYPE', 'LAB_ORDER_DATE', 'SPECIMEN_DATE',
       'SPECIMEN_TIME', 'RESULT_DATE', 'RESULT_TIME', 'RESULT_QUAL',
       'RESULT_SNOMED', 'RESULT_NUM', 'RESULT_MODIFIER', 'RESULT_UNIT',
       'NORM_RANGE_LOW', 'NORM_MODIFIER_LOW', 'NORM_RANGE_HIGH',
       'NORM_MODIFIER_HIGH', 'ABN_IND', 'SOURCE_masked', 'BIRTH_DATE', 'SEX',
       'HISPANIC', 'RACE', 'ZIP_CODE', 'index_date', 'index_age',
       'study_range_min', 'study_range_max', 'study_range_max_1',
       'study_range_max_05', 'DEATH_DATE', 'IF_any3cancer', 'lab_age',
       'age_band', 'BreastCancer', 'ColorectalCancer', 'LungCancer'],
      dtype='object')


# Remove records without LOINC_CODE

In [5]:
#missing LOINC code counts
print(sum(lab_result_cm_paired['LAB_LOINC'].isna()), ' out of ', len(lab_result_cm_paired['LAB_LOINC']), ' rows of LAB table miss LOINC code information')
print('in percentage, is ',np.round(np.mean(lab_result_cm_paired['LAB_LOINC'].isna())*100,2),'%')

988747  out of  76972453  rows of LAB table miss LOINC code information
in percentage, is  1.28 %


In [6]:
#exclude rows with missing LOINC code info
lab_result_cm_paired = lab_result_cm_paired.loc[~lab_result_cm_paired['LAB_LOINC'].isna()]
print(len(lab_result_cm_paired['LAB_LOINC']))

75983706


# Store processed data

In [175]:
pickle.dump(lab_result_cm_paired, open('/data/datasets/changyuyin/1_Personalized_Lab_Test/lab_result_cm_processed.pkl','wb'))
pickle.dump(death_date, open('/data/datasets/changyuyin/1_Personalized_Lab_Test/death_date_processed.pkl','wb'))

In [145]:
pickle.dump(lung_patient, open('/data/datasets/changyuyin/1_Personalized_Lab_Test/lung_patient.pkl','wb'))
pickle.dump(colorectal_patient, open('/data/datasets/changyuyin/1_Personalized_Lab_Test/colorectal_patient.pkl','wb'))
pickle.dump(breast_patient, open('/data/datasets/changyuyin/1_Personalized_Lab_Test/breast_patient.pkl','wb'))